In [24]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage
from langchain_core.prompts import ChatPromptTemplate
from typing import List
from langchain_core.tools import tool

In [15]:
# MODEL = 'llama3.2:1b'
MODEL = 'llama3.1'
llm = ChatOllama(
    model = MODEL,
    temperature = 0.0,
    )

In [16]:
messages = [
    ('system', 'You are a helpful assistant that translates English to French. Translate the user sentence'),
    ('human', 'I love programming')
]

ai_msg = llm.invoke(messages)
print(ai_msg.content)

The translation of "I love programming" in French is:

"J'adore programmer."


In [23]:
prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'You are a helpful assistant that translates {input_language} to {output_language}. Keep output sad'),
        ('human', '{input}')
    ]
)

chain = prompt | llm

msg = chain.invoke(
    {
        'input_language': "English",
        'output_language': "German",
        "input": "I love programming"
    }
)

print(msg.content)

Ich liebe Programmieren... (I love programming...) *sigh* If only I had more time for it, but life is just so... complicated...


# Using tools

In [27]:
@tool
def validate_user(user_id: int, addresses: List[str]) -> bool:
    '''Validate user using historical addresses'''
    return True

llm = llm.bind_tools([validate_user])

res = llm.invoke("Could you validate user 123? They used to live at 123 Fake St in Boston MA")

print(res.tool_calls)


[{'name': 'validate_user', 'args': {'addresses': '[{"street":"Fake St","city":"Boston","state":"MA"}]', 'user_id': 123}, 'id': 'a57baed0-b9af-41a5-8382-475b88f1b769', 'type': 'tool_call'}]


# Embedding

https://ollama.com/blog/embedding-models

- mxbai-embed-large, params: 334M
- nomic-embed-text, params: 137M
- all-minilm, params: 23M


In [28]:
import ollama
import chromadb

In [32]:
documents = [
  "Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
  "Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands",
  "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall",
  "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
  "Llamas are vegetarians and have very efficient digestive systems",
  "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old",
]

for d in documents:
    response = ollama.embeddings(model="mxbai-embed-large", prompt=d)
    emb =response['embedding']
    print(type(emb))
    print(len(emb))
    break

<class 'list'>
1024


In [33]:
# Using Langchain-nomic instead

